# ACRRU RAG Model Testing 
Autonomous Climate Resiliency Research Unit

In [1]:
import pandas as pd
import gspread
import time
from typing import Optional, Type, Union

from langchain import hub

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.utilities.tavily_search import TavilySearchAPIWrapper

from langchain.tools import BraveSearch, BaseTool, StructuredTool, tool

from langchain_openai import ChatOpenAI

from langchain.agents import AgentExecutor
from langchain.agents import create_openai_tools_agent

from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, PromptTemplate, chat
from langchain.schema import AIMessage, HumanMessage, SystemMessage

from langchain.pydantic_v1 import BaseModel, Field


C:\Users\14102\AppData\Local\Temp\ipykernel_19608\1531280756.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Define some constants 

In [2]:
# Controls how much info is provided during a run
VERBOSE = True

# Controls what OpenAI model to use
OPENAI_MODEL = "gpt-4-turbo-preview"

# Controls degree of randomness in response. Scale is from 0 to 2
MODEL_TEMP = 0.0

# Controls whether to output research plan steps to the user along with final response
RETURN_INTERMEDIATE_STEPS = True

# API key for the Brave web search tool
BRAVE_API_KEY = 'BSAmw298lpAtPuiP5crDrAtOpGPgf5U'

# Filepath to Google Sheets credentials file for automated logging
GSHEETS_ACCT_CRED_PATH = 'C:\\Users\\14102\\Brown\\Internships\\INL\\Auth\\acrru-output-logs-97349d3b2c0b.json'

## Utility Functions for Logging Outputs to a Google Sheet

In [3]:
# Connect to google sheets

gc = gspread.service_account(filename=GSHEETS_ACCT_CRED_PATH)

sh = gc.open("ACRRU Output Logs")

def append_to_sheet(data: list, sheet_name: str = "ACRRU Output Logs", tab_name: Union[str, int] = 0) -> None:
    """
    Helper function which appends a list of values to the next available row of a specified Google Sheets tab.
    """
    sheet = gc.open(sheet_name)
    sheet_tab = sheet.get_worksheet(tab_name)

    if isinstance(data[0], list):
        for row in data:
            sheet_tab.append_row(row)
    
    else:
        sheet_tab.append_row(data)


# Collect data to log 
def collect_log_data(agent, agent_output, test_input_dict: dict, target_cap: int, notes: str) -> list:
    """
    Compiles all information about a provided ACRRU research output to be logged into a list. 
    """
    # Get timestamp
    cur_time = time.ctime(time.time())
    # Get intermediate queries the model called itself
    int_steps = '\n\n'.join([int_step[0].log.strip() for int_step in agent_output['intermediate_steps']])
    # Get the input template
    template = '\n'.join([f'{input_value.content}' for input_value in agent.steps[1].format_messages(**test_input_dict, agent_scratchpad=['']) if input_value.__class__.__name__ == 'SystemMessage'])
    # Get the input prompt
    prompt = '\n'.join([f'{input_value.content}' for input_value in agent.steps[1].format_messages(**test_input_dict, agent_scratchpad=['']) if input_value.__class__.__name__ == 'HumanMessage'])

    # Return a list of everything!
    return [cur_time, test_input_dict['org_name'], target_cap, notes, 
           template, prompt, int_steps, agent_output['output']]

## Compile Electricity Providers

Data from [EIA 2022 report](https://www.eia.gov/electricity/data/eia861/)

In [4]:
# Get list of electricity providers
elec_industry_df = pd.read_csv("C:\\Users\\14102\\Brown\\Internships\\INL\\Data\\Frame_2022.csv")
# Get list of the territory electricity providers service. At the State/County level.
service_territory_df = pd.read_csv("C:\\Users\\14102\\Brown\\Internships\\INL\\Data\\Service_Territory_2022.csv")

service_territory_df = service_territory_df.drop(
    [col for col in service_territory_df.columns if col not in ['Utility Number', 'State', 'County']], axis=1).reset_index(drop=True)

In [5]:
# Only look at investor owned companies for now!
investor_utils = elec_industry_df[elec_industry_df['Ownership'] == 'Investor Owned']
# Drop some columns
investor_utils = investor_utils.drop(
    [col for col in investor_utils.columns if col not in ['Utility Number', 'Utility Name']], axis=1).reset_index(drop=True)

In [6]:
# Join investor owned utils with their reported coverage
investor_util_coverage = investor_utils.merge(service_territory_df, on='Utility Number', how='inner')

In [7]:
investor_util_coverage

,Utility Number,Utility Name,State,County
0,176,Ajo Improvement Co,AZ,Pima
1,195,Alabama Power Co,AL,Autauga
2,195,Alabama Power Co,AL,Baldwin
3,195,Alabama Power Co,AL,Barbour
4,195,Alabama Power Co,AL,Bibb
...,...,...,...,...
3555,60631,Upper Michigan Energy Resources Corp.,MI,Houghton
3556,60631,Upper Michigan Energy Resources Corp.,MI,Iron
3557,60631,Upper Michigan Energy Resources Corp.,MI,Marquette
3558,60631,Upper Michigan Energy Resources Corp.,MI,Menominee


## Build Web Searching Tool (Tavily) for Agent

[Tavily link](https://tavily.com/)

[Usage in LangChain](https://python.langchain.com/docs/integrations/tools/tavily_search)

In [8]:
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search, max_results = 4)

In [22]:
# response = tavily_tool.invoke({'query': 'Baltimore Gas and Electric climate resilience executive leadership'})

## Build Backup Search Tool

[Usage in LangChain](https://python.langchain.com/docs/integrations/tools/brave_search)

In [14]:
brave_tool = BraveSearch.from_api_key(api_key=BRAVE_API_KEY, search_kwargs={"count": 4}, description='a search engine. useful to use when other tools do not return results.')

## Get CRMM Text file 

In [9]:
# Full CRMM file with all information
with open('crmm_docs/CRMM_13.txt', "r", encoding='utf-8') as f:
    crmm_raw = f.read()

# Summarized version of CRMM doc (done with ChatGPT)
with open('crmm_docs/CRMM_AI_Summary.txt', "r", encoding='utf-8') as f:
    crmm_condensed = f.read()

# Full CRMM file without the capability section
with open('crmm_docs/CRMM_header.txt', "r", encoding='utf-8') as f:
    crmm_head = f.read()

cap_dict = {}
# CRMM Capability descriptions
for num in range(5):
    with open(f'crmm_docs/Capability {num+1}.txt', "r", encoding='utf-8') as f:
        cap = f.read()
    cap_dict[f'cap{num+1}'] = cap

cap_dict['full'] = crmm_raw

## Build Chat Prompt for Agent

[LangChain Documentation](https://python.langchain.com/docs/modules/model_io/prompts/quick_start)

Prompt for all-capability evaluation/research

In [10]:
# OpenAI agents need placeholders for chat history and agent scratchpad.
chat_prompt = ChatPromptTemplate.from_messages(
                # System message gives the model the 'backstory' for the task (eg. how do you act? what info do you have on hand? what are you designed to do?), but does not actually respond to this directly
                [SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['crmm_info'], 
                    template="You are an analyst who uses the Climate Resilience Maturity Model to summarize how infrastructure providers are implementing climate resiliency measures." 
                     "\nYou are critical of providers, and only give them credit for progress in a capability if specific and direct evidence is provided related to infrastructure development or resilience planning and organization." 
                     " The Climate Resilience Maturity Model is explained below:\n####\n{crmm_info}\n####\n")),
                chat.MessagesPlaceholder(variable_name='chat_history', optional=True),
                # Human message represents the question actually being asked by a user or service. The model's output is a response to this message.
                chat.HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['org_name'], template="Summarize how the infrastructure and initiatives of the organization, {org_name} is positioned with respect to the Climate Resilience Maturity Model. "
                     "For each capability, provide a one sentence summary, list all the evidence that led to the conclusion, and cite sources. All evidence MUST be directly related to the Climate Resilience Maturity Model. Use as many different relevant sources as possible to gauge their overall position. \n"
                     "**Research Approach**: \n 1. Search for any documents related to {org_name} and climate resilience. \n 2. Search for documents specifically related to capabilities not accounted for in step 1. \n"
                     " Do **not** consider the following topics when collecting evidence:\n -greenhouse gas emissions \n -energy saving and efficiency \n -decarbonization \n -clean energy")),
                chat.MessagesPlaceholder(variable_name='agent_scratchpad')])

Prompt for capability-specific evaluation/research

In [11]:
# OpenAI agents need placeholders for chat history and agent scratchpad.
chat_prompt_cap = ChatPromptTemplate.from_messages(
                # System message gives the model the 'backstory' for the task (eg. how do you act? what info do you have on hand? what are you designed to do?), but does not actually respond to this directly
                [SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['crmm_info'], 
                    template="You are an analyst who uses the Climate Resilience Maturity Model to summarize how infrastructure providers are implementing climate resiliency measures." 
                     "\nYou are critical of providers, and only give them credit for progress in a capability if specific and direct evidence is provided related to infrastructure development or resilience planning and organization." 
                     " The Climate Resilience Maturity Model is explained below:\n####\n{crmm_info}\n")),
                chat.MessagesPlaceholder(variable_name='chat_history', optional=True),
                # Human message represents the question actually being asked by a user or service. The model's output is a response to this message.
                chat.HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['org_name', 'cap_info'], template="Summarize how the infrastructure and initiatives of the organization, {org_name} are positioned with respect to the following capability in the Climate Resilience Maturity Model: \n"
                     "####\n{cap_info}\n####\n"
                     "For the capability, provide a summary for levels 1, 2, and 3, list all evidence, and cite all sources used. All evidence MUST be related to climate resiliency in some way. Use as many different relevant sources as possible to gauge their overall position. \n"
                     "Do not include an introduction or conclusion in your response. \nDo not consider the following topics when collecting evidence:\n -greenhouse gas emissions \n -energy saving and efficiency \n -decarbonization \n -clean energy")),
                chat.MessagesPlaceholder(variable_name='agent_scratchpad')])

## Create the Agent

[LangChain Agent Documentation](https://python.langchain.com/docs/modules/agents/quick_start)

[LangChain Tools Documentation](https://python.langchain.com/docs/modules/agents/tools/)

In [12]:
# Define the model (GPT-4)
llm = ChatOpenAI(model=OPENAI_MODEL, temperature=MODEL_TEMP)

In [15]:
# Define tools available to agent
tools = [brave_tool, tavily_tool]

# Use this function to make agents based on OpenAI models
agent = create_openai_tools_agent(llm, tools, chat_prompt_cap)

In [16]:
# Define agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=VERBOSE, return_intermediate_steps=RETURN_INTERMEDIATE_STEPS)

## Running the Agent Iteratively

### Agent run across multiple providers, **assessing all capabilities simultaneously** per organization

In [ ]:
# Define input list: Currently is investor-owned elec utils from MD
md_elec_providers = investor_util_coverage[investor_util_coverage['State'] == 'MD']['Utility Name'].unique()
research_output = {}
# Iterate over every provider in our array
for organization in md_elec_providers:

    ## Compile Agent Input
    input_dict = {
        'crmm_info': crmm_raw,
        'org_name': organization}

    # Run the agent for given org
    agent_output: dict = agent_executor.invoke(input_dict)

    # Record result in dict
    research_output[organization] = agent_output['output']

In [ ]:
rag_output = pd.DataFrame.from_dict(research_output, orient='index', columns=['Research Results']).reset_index(names=['Utility Name'])
rag_output = rag_output.merge(investor_utils, on='Utility Name', how='inner')

## Observing Outputs

In [ ]:
for org, rag_response in research_output.items():
    print(f'RAG Response for {org}:\n')
    print(rag_response)
    print('\n\n\n')

### ACRRU run across one provider, **assessing each capability individually**

In [17]:
# For an example, lets see what electric utility providers serve DC!
investor_util_coverage[investor_util_coverage['State'] == 'DC']['Utility Name'].unique()

array(['Potomac Electric Power Co'], dtype=object)

In [20]:
# Iterate over all capabilities, and skip over the full crmm item.
org_name = 'Potomac Electric Power Co'

# Custom notes 
notes = "Running ACRRU with significantly condensed CRMM document. I summarized the non-capability part via ChatGPT 3.5. Also more explicitly describing the desired output structure."
for target_cap, info in cap_dict.items():
    # Skip the full CRMM input.
    if target_cap == 'full':
        continue

    # Define input dict for ACRRU
    test_input_dict = {
        # Lets test with a much shorter version of CRMM
    'crmm_info': crmm_condensed, #crmm_head,
    'org_name': org_name,
    'cap_info': info}
    
    # Actually runs ACRRU! Returns a dict of inputs (prompt + template) and output
    agent_output = agent_executor.invoke(test_input_dict)

    # Compile data to log
    log_data = collect_log_data(agent, agent_output, test_input_dict, target_cap, notes)

    # Log data in google sheet
    append_to_sheet(data=log_data)



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Potomac Electric Power Co executive climate resilience lead'}`


[{'url': 'https://www.utilitydive.com/news/as-pepco-outlines-plan-to-meet-dc-climate-goals-consumer-advocates-say-equ/603673/', 'content': 'Get the free newsletter\nSubscribe to Utility Dive for top news, trends & analysis\nMost Popular\nLibrary resources\nCompany Announcements\nWhat We\'re Reading\nEvents\nIndustry Intel\nUtility Dive news delivered to your inbox\nGet the free daily newsletter read by industry experts\nCompany Announcements\nDistributed Energy Resource Growth\nDistributed energy resources, including rooftop solar, battery storage and electric vehicles, are experiencing significant growth in the U.S. as the power sector evolves to a cleaner, less centralized future.\n In partnership with\nTopics covered: Solar technology, business models, regulations and policy, distributed solar, utility solar\nIn partnership

## Custom Search Tool with Backup (WIP)

[LangChain Documentation](https://python.langchain.com/docs/modules/agents/tools/custom_tools)

In [ ]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")


class CustomSearchTool(BaseTool):
    name = "custom_search"
    description = "useful for when you need to answer questions using the internet."
    args_schema: Type[BaseModel] = SearchInput

    def __init__(self, tavily, brave): 
        super(BaseTool, self).__init__()
        self.tavily = tavily
        self.brave = brave

    def _run(
        self, query: str) -> str:
        """Use the tool."""
        output = self.tavily.run(query)
        if not output:
            output = self.brave.run(query)
        
        return output

    async def _arun(
        self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")


In [ ]:
custom_search = CustomSearchTool(tavily_tool, brave_tool)